In [1]:
import torch
import eigenpro.kernels as kernels
import eigenpro.models.sharded_kernel_machine as skm
import eigenpro.solver as solver
import eigenpro.utils.device as dev

In [2]:
n, p, d, c = 500, 100, 5, 2

Z = torch.randn(p, d)
X_train, X_test = torch.randn(n//2, d), torch.randn(n//2, d)
W_star = torch.randn(d, c)
Y_train, Y_test = X_train @ W_star, X_test @ W_star

dtype = torch.float32
kernel_fn = lambda x, z: kernels.laplacian(x, z, bandwidth=20.)
device = dev.Device.create(use_gpu_if_available=True)
model = skm.create_sharded_kernel_machine(
    Z, c, kernel_fn, device, dtype=dtype, tmp_centers_coeff=2)

sd, sm, qd, qm = 10, 10, 3, 3 # configuration for EigenPro preconditioners
model = solver.fit(model, X_train, Y_train, X_test, Y_test, device,
                   dtype=dtype, kernel=kernel_fn, n_data_pcd_nyst_samples=sd, n_model_pcd_nyst_samples=sm,
                   n_data_pcd_eigenvals=qd, n_model_pcd_eigenvals=qm, epochs=2, accumulated_gradients=True)

notice: the current implementation can only support 1 GPU, we only use the following device: (cuda:0) 


TypeError: fit() got an unexpected keyword argument 's_data'